<a href="https://colab.research.google.com/github/ChiliJoe/genai-intro/blob/main/ChatGPT_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an expansion of tutorial from https://youtu.be/0-zlUy7VUjg using real-world API from openweathermap.org.

Let's use the openweathermap current weather API that requires geo code as input.

In [12]:
%%capture
# Set openweathermap.org API key
%env OWM_API_KEY=

# Set OpenAI API key
%env OPENAI_API_KEY=

In [13]:
%%capture
!rm -rf helper
!git clone https://github.com/ChiliJoe/genai-intro.git helper

from helper.weather import function_list

# Vanilla OpenAI

In [14]:
%%capture
!pip install openai

In [15]:
from IPython.display import display, Markdown
from helper.demolib import Color

import openai
import os
import json

openai.api_key  = os.getenv('OPENAI_API_KEY')

class Chat:
  def __init__(self, system_message, function_list):
    self.messages = [{"role": "system", "content": system_message}]
    self.function_list = function_list
    self.function_descriptions = [f["description"] for f in function_list.values()]

  def ask(self, role_content, role="user", function_name=None):
    if role == "function":
      self.messages.append({"role": role, "name": function_name, "content": role_content})
    else:
      self.messages.append({"role": role, "content": role_content})

    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=self.messages,
        functions=self.function_descriptions,
        function_call="auto"
    )
    ai_message = res.choices[0].message
    self.messages.append(ai_message)

    # process function call
    if ai_message["content"] is None and ai_message["function_call"]:
      target_function_name = ai_message["function_call"]["name"]
      target_function = self.function_list[target_function_name]["function"]
      func_params = json.loads(ai_message['function_call']['arguments'])

      print(Color.cyan(f"Invoking {target_function_name} with parameters {func_params}"))
      function_response = target_function(**func_params)
      print(Color.white("Function response: "), end="")
      print(Color.magenta(f"{function_response}"))

      self.ask(role_content=json.dumps(function_response),
               role="function",
               function_name=target_function_name
               )

    elif ai_message["content"]:
      print(Color.green("===== Final response: ====="))
      display(Markdown(ai_message["content"]))

In [16]:
sysmessage="""
You are a helpful AI chatbot.
"""
chat = Chat(sysmessage, function_list)

In [17]:
chat.ask("What's the weather like in the White House?")

Invoking get_location_data with parameters {'city': 'Washington', 'country': 'US'}
Function response: [{'name': 'Washington', 'local_names': {'ml': 'വാഷിങ്ടൺ, ഡി.സി.', 'ba': 'Вашингтон', 'ru': 'Вашингтон', 'gl': 'Washington, D.C.', 'ms': 'Washington, D.C.', 'de': 'Washington', 'ur': 'واشنگٹن', 'te': 'వాషింగ్టన్, డి.సి.', 'lb': 'Washington D.C.', 'mi': 'Wahingitana', 'qu': 'Washington DC', 'lt': 'Vašingtonas', 'el': 'Ουάσιγκτων', 'cs': 'Washington, D.C.', 'sl': 'Washington, D.C.', 'bg': 'Вашингтон', 'ks': 'واشنگٹن', 'ts': 'Washington, DC', 'ja': 'ワシントン', 'sq': 'Uashington D.C.', 'bs': 'Washington, D.C.', 'ar': 'واشينطون', 'kw': 'Washington, D.C.', 'ko': '워싱턴', 'yo': 'Washington, D.C.', 'gv': 'Washington, D.C.', 'kk': 'Вашингтон', 'sn': 'Washington, DC', 'pi': 'वासिंगटन डि सि', 'xh': 'Washington, D.C.', 'si': 'වොෂින්ටන්, ඩී.සී.', 'ca': 'Washington DC', 'uk': 'Вашингтон', 'ln': 'Washington', 'ku': 'Washington', 'eu': 'Washington', 'ro': 'Washington, D.C.', 'et': 'Washington', 'fy': 'Washi

The weather in Washington D.C., near the White House, is currently overcast with a temperature of 30.01 degrees Celsius. It feels like 35.31 degrees Celsius. The minimum temperature is 27.81 degrees Celsius, and the maximum temperature is 32.29 degrees Celsius. The humidity level is 71%, and there is a wind speed of 5.66 m/s coming from the southeast.